In [ ]:
import plotly.graph_objects as go

import pandas as pd
import numpy as np
import time
import os
import requests
from pprint import pprint
import subprocess
import datetime as dt
import json
import pandas as pd

from xgboost import XGBClassifier
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/hajsy'

Mounted at /content/drive
/content/drive/My Drive/hajsy


In [ ]:
btc_df = pd.read_csv('data_jan_oct.csv', index_col=0) # BTC JAN TO OCT
# doge_df = pd.read_csv('data_doge_jan_sep.csv', index_col=0) # DOGE JAN TO SEPT

In [ ]:
btc_df

,open,high,low,close,vol
2021-01-01 00:00:00,28923.63,28961.66,28913.12,28961.66,27.457032
2021-01-01 00:01:00,28961.67,29009.91,28961.01,29017.50,58.477501
2021-01-01 00:02:00,29009.54,28989.30,28973.58,29016.71,42.470329
2021-01-01 00:03:00,28989.68,28982.69,28972.33,28999.85,30.360677
2021-01-01 00:04:00,28982.67,28975.65,28971.80,28995.93,24.124339
...,...,...,...,...,...
2021-10-24 20:35:00,60973.73,60990.30,60942.99,61010.79,26.955470
2021-10-24 20:36:00,60990.29,60988.67,60967.21,60996.24,15.450520
2021-10-24 20:37:00,60988.67,60939.90,60939.90,60988.67,39.582950
2021-10-24 20:38:00,60939.90,60939.99,60928.71,60950.00,11.011360


## bollinger bands strategy tests with few tweaks

In [ ]:
def test_strategy_bollinger_down(prices_df, period_len = 100, size_std = 3, ma_std = 0, cross_col='cross_signal_down_down', cross_ma_col='cross_ma_down', ret_plot = False, eps=0.0):
    
    prices_df["ma"] = prices_df.price.rolling(period_len).mean() + ma_std*prices_df.price.rolling(period_len).std()# 5h hourse
    prices_df["std"] = prices_df.price.rolling(period_len).std()
    prices_df["bb_u"] = prices_df["ma"] + size_std*prices_df["std"]
    prices_df["bb_l"] = prices_df["ma"] - size_std*prices_df["std"]

    prices_df["cross_signal_down_down"] = (prices_df.price.shift(1) > prices_df.bb_l.shift(1)) & (prices_df.price < prices_df.bb_l)
    prices_df["cross_signal_down_up"] = (prices_df.price.shift(1) < prices_df.bb_l.shift(1)) & (prices_df.price > prices_df.bb_l)
    prices_df["cross_signal_up_up"] = (prices_df.price.shift(1) < prices_df.bb_u.shift(1)) & (prices_df.price > prices_df.bb_u)
    prices_df["cross_signal_up_down"] = (prices_df.price.shift(1) > prices_df.bb_u.shift(1)) & (prices_df.price < prices_df.bb_u)
    prices_df['cross_ma_down'] = (prices_df.price.shift(1) < prices_df.ma.shift(1)) & (prices_df.price > prices_df.ma)
    prices_df['cross_ma_up'] = (prices_df.price.shift(1) > prices_df.ma.shift(1)) & (prices_df.price < prices_df.ma)
    cross_down = np.where(prices_df[cross_col])[0]
    cross_ma = np.where(prices_df[cross_ma_col])[0] 

    money = 1000
    stock = 0
    num_trans = 0
    have_stock = False
    prev_price = 0
    sells = []
    buys = []
    moneys = []

    for i in range(prices_df.shape[0]):
        prev_state = have_stock
        money, stock, have_stock, prev_price = bollinger_strategy_down( prices_df.price[i], prev_price , money, stock, have_stock, i in cross_ma, i in cross_down, eps)

        if have_stock != prev_state:
            num_trans+=1
            if have_stock:
                buys.append(i)
            else:
                sells.append(i)
                moneys.append(money)
            
    sells = np.array(sells)
    buys = np.array(buys)
    moneys = np.array(moneys)
    #####

    if ret_plot:

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=prices_df.index, y=prices_df.price, mode="lines", name="price"))
        fig.add_trace(go.Scatter(x=prices_df.index, y=prices_df.ma, mode="lines", name=f"ma period_len {period_len}"))
        fig.add_trace(go.Scatter(x=prices_df.index[buys], y=prices_df.price[buys], mode="markers", marker=dict(size=12, color="green"), name='Buy signal'))
        fig.add_trace(go.Scatter(x=prices_df.index[sells], y=prices_df.price[sells], mode="markers", marker=dict(size=12, color="blue"), name='Sell signal'))

        fig2 = go.Figure()
        fig2.add_trace(go.Scatter(x=prices_df.index[sells], y=moneys, mode="lines", name="price"))

        return money, stock, num_trans, period_len, size_std, ma_std, cross_col, cross_ma_col, sells, buys, fig, fig2

    return money, stock, num_trans, period_len, size_std, ma_std, cross_col, cross_ma_col, sells, buys, eps

def bollinger_strategy_down(price, prev_price, money = 1000, stock = 0, have_stock = False, ma_crossed = False, bollinger_crossed = False, eps = 0.0):
    transaction_fee = 0.999

    if have_stock:
        diff = (prev_price - price) / prev_price

        if ma_crossed or diff > eps:
            money += stock * price * transaction_fee
            stock = 0
            have_stock = False

    elif not have_stock and bollinger_crossed:

        stock = (money / price) * transaction_fee
        prev_price = price
        money = 0
        have_stock = True

    return money, stock, have_stock, prev_price



## few simple test

In [ ]:
x = test_strategy_bollinger_down(btc_df.rename(columns = {'close' : 'price'}), 250,	4.5,	0.3,	'cross_signal_down_down',	'cross_ma_up', ret_plot = True, eps = 0.07)
x[-1].show()

In [ ]:
x = test_strategy_bollinger_down(btc_df.rename(columns = {'close' : 'price'}), 250,	4.5,	0.3,	'cross_signal_down_down',	'cross_ma_up', ret_plot = True, eps = 0.5)
x[-1].show()

In [ ]:
x = test_strategy_bollinger_down(btc_df.rename(columns = {'close' : 'price'}), 350,	3.5,	0.3,	'cross_signal_down_down',	'cross_ma_up', ret_plot = True, eps = 0.5)
x[-1].show()

In [ ]:
def test_strategies_bollinger_down(df, prices_df, period_len=300, size_std=3, ma_std=0, eps = 0.1):


  params1 = test_strategy_bollinger_down(prices_df, period_len=period_len, size_std=size_std, ma_std=ma_std, eps = eps)
  params2 = test_strategy_bollinger_down(prices_df, period_len=period_len, size_std=size_std, ma_std=ma_std, cross_col='cross_signal_down_up', eps = eps)
  params3 = test_strategy_bollinger_down(prices_df, period_len=period_len, size_std=size_std, ma_std=ma_std, cross_ma_col='cross_ma_up', eps = eps)
  params4 = test_strategy_bollinger_down(prices_df, period_len=period_len, size_std=size_std, ma_std=ma_std, cross_col='cross_signal_down_up', cross_ma_col='cross_ma_up', eps = eps)


  for p in [params1, params2, params3, params4]:
    p = pd.Series(p, index = df.columns)
    df = df.append(p, ignore_index=True)
  return df


## rebuild for bot - not tested

In [ ]:
class BollingerSignalManagerExtended(SignalManager):
  #I assume more argument to function are possible
    def __init__(self, n: int, stdlevel: int, ma_std_level: int, cross_bb: string, cross_ma: string, have_stock, stop_loss):
        self.n = n
        self._stdlevel = stdlevel
        self._ma_std_level = ma_std_level
        self._cross_bb = cross_bb
        self._cross_ma = cross_ma

        self._have_stock = have_stock
        self._stop_loss = stop_loss

        self._last_buy_price = 0
        self._last_sell_price = 0

    def enrich_context(self):
        #we calculate technical indicators based on close and perhaps vol
        self.context_df["ma"] = self.context_df.close.rolling(self._n).mean() + ma_std_level * context_df.close.rolling(self._n).std()
        self.context_df["mstd"] = self.context_df.close.rolling(self._n).std()
        self.context_df["bb_bottom"] = self.context_df.ma - self._stdlevel * self.context_df.mstd

    def signal(self):

        if not self._have_stock:
          if self._cross_bb == 'down':
            if (self.context_df.iloc[-1].close > self.context_df.iloc[-1].bb_bottom ) and (self.context_df.iloc[-2].close < self.context_df.iloc[-2].bb_bottom):
              return SimpleSignal.ENTER
          
          elif self._cross_bb == 'up':
            if (self.context_df.iloc[-1].close < self.context_df.iloc[-1].bb_bottom ) and (self.context_df.iloc[-2].close > self.context_df.iloc[-2].bb_bottom):
              return SimpleSignal.ENTER


        if self._have_stock:
          if (self._last_buy_price - self.context_df.iloc[-1].close) / self._last_buy_price > self._stop_loss:
            return SimpleSignal.EXIT

          elif self._cross_ma == 'down':
            if (self.context_df.iloc[-1].close > self.context_df.iloc[-1].ma ) and (self.context_df.iloc[-2].close < self.context_df.iloc[-2].ma):
              return SimpleSignal.EXIT
          
          elif self._cross_ma == 'up':
            if (self.context_df.iloc[-1].close < self.context_df.iloc[-1].ma ) and (self.context_df.iloc[-2].close > self.context_df.iloc[-2].ma):
              return SimpleSignal.EXIT

        else:
            return SimpleSignal.NOTHING